In [13]:
!pip install --upgrade pip
!pip install --quiet flashtext==2.7
!pip install git+https://github.com/boudinfl/pke.git
!pip install tokenizers
!pip install sentencepiece==0.1.97
!pip install --no-dependencies transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-huh04q67
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-huh04q67
  Resolved https://github.com/boudinfl/pke.git to commit ebd6e5754b4156a61a4ec6c4c283e821d11a36be
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 2.9.0 requires tokenizers==0.7.0, but you have tokenizers 0.13.3 which is incompatible.
Looking in in

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
import json
import requests
import string
import re
import nltk
import string
import itertools
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
# import pke
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import traceback
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
import textwrap

text = """There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pulled into the mantle, forming magma that erupts as volcanoes. When a line of volcanoes forms along a subduction zone, they make up a volcanic arc. The edges of the Pacific plate are long subduction zones lined with volcanoes. This is why the Pacific rim is called the “Pacific Ring of Fire.”"""

wrapper = textwrap.TextWrapper(width=150)
word_list = wrapper.wrap(text=text)
for element in word_list: 
  print(element) 

There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic
Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other
at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s
surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental
crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This
makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate
is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pu

In [17]:
import pke
def get_keywords(text):
    out=[]
    try:
        extractor = pke.unsupervised.YAKE()
        extractor.load_document(input=text,language='en')
        grammar = r"""
                NP:
                    {<NOUN|PROPN>+}
            """
        extractor.ngram_selection(n=1)
        extractor.grammar_selection(grammar=grammar)
        extractor.candidate_selection(n=1)

        extractor.candidate_weighting(window=3,
                                      use_stems=False)

        keyphrases = extractor.get_n_best(n=30)
        

        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

keywords = get_keywords(text)[:8]
print ("keywords: ",keywords)

keywords:  ['plate', 'crust', 'volcanoes', 'divergent', 'pacific', 'rock', 'boundaries', 'molten']


In [18]:
sentences = tokenize_sentences(text)
print (sentences)

['There is a lot of volcanic activity at divergent plate boundaries in the oceans.', 'For example, many undersea volcanoes are found along the Mid-Atlantic Ridge.', 'This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean.', 'As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust.', 'Molten rock, called magma, erupts through these cracks onto Earth’s surface.', 'At the surface, the molten rock is called lava.', 'It cools and hardens, forming rock.', 'Divergent plate boundaries also occur in the continental crust.', 'Volcanoes form at these boundaries, but less often than in ocean crust.', 'That’s because continental crust is thicker than oceanic crust.', 'This makes it more difficult for molten rock to push up through the crust.', 'Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone.', 'The leadi

In [19]:
from pprint import pprint
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=False)
        keyword_sentences[key] = values
    return keyword_sentences

keyword_sentence_mapping = get_sentences_for_keyword(keywords, sentences)
pprint (keyword_sentence_mapping)

{'boundaries': ['Divergent plate boundaries also occur in the continental '
                'crust.',
                'Volcanoes form at these boundaries, but less often than in '
                'ocean crust.',
                'There is a lot of volcanic activity at divergent plate '
                'boundaries in the oceans.',
                'Many volcanoes form along convergent plate boundaries where '
                'one tectonic plate is pulled down beneath another at a '
                'subduction zone.'],
 'crust': ['Divergent plate boundaries also occur in the continental crust.',
           'That’s because continental crust is thicker than oceanic crust.',
           'That’s because continental crust is thicker than oceanic crust.',
           'Volcanoes form at these boundaries, but less often than in ocean '
           'crust.',
           'This makes it more difficult for molten rock to push up through '
           'the crust.',
           'As tectonic plates pull away f

###Bert

In [20]:
import os
import zipfile

bert_wsd_pytorch = "/content/gdrive/My Drive/bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6.zip"
extract_directory = "/content/gdrive/My Drive"

extracted_folder = bert_wsd_pytorch.replace(".zip","")

#  If unzipped folder exists don't unzip again.
if not os.path.isdir(extracted_folder):
  with zipfile.ZipFile(bert_wsd_pytorch, 'r') as zip_ref:
      zip_ref.extractall(extract_directory)
else:
  print (extracted_folder," is extracted already")

/content/gdrive/My Drive/bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6  is extracted already


###Run model

In [21]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [22]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [23]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import torch
import math
from transformers import BertModel, BertConfig, BertPreTrainedModel, BertTokenizer

class BertWSD(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)

        self.ranking_linear = torch.nn.Linear(config.hidden_size, 1)

        self.init_weights()

    

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dir = "/content/gdrive/My Drive/bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6"


model = BertWSD.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)

if '[TGT]' not in tokenizer.additional_special_tokens:
    tokenizer.add_special_tokens({'additional_special_tokens': ['[TGT]']})
    assert '[TGT]' in tokenizer.additional_special_tokens
    model.resize_token_embeddings(len(tokenizer))
    
model.to(DEVICE)
model.eval()

BertWSD(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30523, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [25]:
import csv
import os
from collections import namedtuple

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

import torch
import re
import time
import torch
from tabulate import tabulate
from torch.nn.functional import softmax
from tqdm import tqdm
from transformers import BertTokenizer

GlossSelectionRecord = namedtuple("GlossSelectionRecord", ["guid", "sentence", "sense_keys", "glosses", "targets"])
BertInput = namedtuple("BertInput", ["input_ids", "input_mask", "segment_ids", "label_id"])

MAX_SEQ_LENGTH = 128

def _create_features_from_records(records, max_seq_length, tokenizer, cls_token_at_end=False, pad_on_left=False,
                                  cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                  sequence_a_segment_id=0, sequence_b_segment_id=1,
                                  cls_token_segment_id=1, pad_token_segment_id=0,
                                  mask_padding_with_zero=True, disable_progress_bar=False):
    
    features = []
    for record in tqdm(records, disable=disable_progress_bar):
        tokens_a = tokenizer.tokenize(record.sentence)

        sequences = [(gloss, 1 if i in record.targets else 0) for i, gloss in enumerate(record.glosses)]

        pairs = []
        for seq, label in sequences:
            tokens_b = tokenizer.tokenize(seq)

            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)

            tokens = tokens_a + [sep_token]
            segment_ids = [sequence_a_segment_id] * len(tokens)

            tokens += tokens_b + [sep_token]
            segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

            if cls_token_at_end:
                tokens = tokens + [cls_token]
                segment_ids = segment_ids + [cls_token_segment_id]
            else:
                tokens = [cls_token] + tokens
                segment_ids = [cls_token_segment_id] + segment_ids

            input_ids = tokenizer.convert_tokens_to_ids(tokens)

            input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

            padding_length = max_seq_length - len(input_ids)
            if pad_on_left:
                input_ids = ([pad_token] * padding_length) + input_ids
                input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
                segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            else:
                input_ids = input_ids + ([pad_token] * padding_length)
                input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
                segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            pairs.append(
                BertInput(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_id=label)
            )

        features.append(pairs)

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
from pprint import pprint
nltk.download('omw-1.4')
sentence = "Mark's favourite game is **Cricket**."

sentence_for_bert = sentence.replace("**"," [TGT] ")
sentence_for_bert = " ".join(sentence_for_bert.split())

print (sentence_for_bert)

re_result = re.search(r"\[TGT\](.*)\[TGT\]", sentence_for_bert)
if re_result is None:
    print("\nIncorrect input format. Please try again.")

ambiguous_word = re_result.group(1).strip()

print ("Word: ",ambiguous_word)


results = dict()

for i, synset in enumerate(set(wn.synsets(ambiguous_word))):
    results[synset] =  synset.definition()

pprint (results)

sense_keys=[]
definitions=[]
for sense_key, definition in results.items():
    sense_keys.append(sense_key)
    definitions.append(definition)


print (sense_keys)
print (definitions)


record = GlossSelectionRecord("test", sentence_for_bert, sense_keys, definitions, [-1])

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Mark's favourite game is [TGT] Cricket [TGT] .
Word:  Cricket
{Synset('cricket.n.01'): 'leaping insect; male makes chirping noises by '
                         'rubbing the forewings together',
 Synset('cricket.n.02'): 'a game played with a ball and bat by two teams of 11 '
                         'players; teams take turns trying to score runs',
 Synset('cricket.v.01'): 'play cricket'}
[Synset('cricket.n.01'), Synset('cricket.n.02'), Synset('cricket.v.01')]
['leaping insect; male makes chirping noises by rubbing the forewings together', 'a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs', 'play cricket']


In [27]:
model.config.vocab_size

30523

In [28]:
features = _create_features_from_records([record], MAX_SEQ_LENGTH, tokenizer,
                                          cls_token=tokenizer.cls_token,
                                          sep_token=tokenizer.sep_token,
                                          cls_token_segment_id=1,
                                          pad_token_segment_id=0,
                                          disable_progress_bar=True)[0]

print (len(features))

for ftr in features:
  print (tokenizer.convert_ids_to_tokens(ftr.input_ids))

3
['[CLS]', 'mark', "'", 's', 'favourite', 'game', 'is', '[UNK]', 'cricket', '[UNK]', '.', '[SEP]', 'leaping', 'insect', ';', 'male', 'makes', 'chi', '##rp', '##ing', 'noises', 'by', 'rubbing', 'the', 'forewings', 'together', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [29]:
with torch.no_grad():
      logits = torch.zeros(len(definitions), dtype=torch.double).to(DEVICE)
      for i, bert_input in list(enumerate(features)):
          logits[i] = model.ranking_linear(
              model.bert(
                  input_ids=torch.tensor(bert_input.input_ids, dtype=torch.long).unsqueeze(0).to(DEVICE),
                  attention_mask=torch.tensor(bert_input.input_mask, dtype=torch.long).unsqueeze(0).to(DEVICE),
                  token_type_ids=torch.tensor(bert_input.segment_ids, dtype=torch.long).unsqueeze(0).to(DEVICE)
              )[1]
          )
      scores = softmax(logits, dim=0)

      preds = (sorted(zip(sense_keys, definitions, scores), key=lambda x: x[-1], reverse=True))

print ("\n")
for pred in preds:
  print (pred)
sense = preds[0][0]
meaning = preds[0][1]

print ("\nMost appropriate sense: ",sense)
print ("Most appropriate meaning: ",meaning)



(Synset('cricket.n.02'), 'a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs', tensor(0.6093, device='cuda:0', dtype=torch.float64))
(Synset('cricket.v.01'), 'play cricket', tensor(0.3907, device='cuda:0', dtype=torch.float64))
(Synset('cricket.n.01'), 'leaping insect; male makes chirping noises by rubbing the forewings together', tensor(9.1672e-06, device='cuda:0', dtype=torch.float64))

Most appropriate sense:  Synset('cricket.n.02')
Most appropriate meaning:  a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs


In [30]:
def get_sense(sent):
  re_result = re.search(r"\[TGT\](.*)\[TGT\]", sent)
  if re_result is None:
      print("\nIncorrect input format. Please try again.")

  ambiguous_word = re_result.group(1).strip()
  results = dict()

  for i, synset in enumerate(set(wn.synsets(ambiguous_word))):
      results[synset] =  synset.definition()

  if len(results) ==0:
    return None

  sense_keys=[]
  definitions=[]
  for sense_key, definition in results.items():
      sense_keys.append(sense_key)
      definitions.append(definition)

  record = GlossSelectionRecord("test", sent, sense_keys, definitions, [-1])

  features = _create_features_from_records([record], MAX_SEQ_LENGTH, tokenizer,
                                            cls_token=tokenizer.cls_token,
                                            sep_token=tokenizer.sep_token,
                                            cls_token_segment_id=1,
                                            pad_token_segment_id=0,
                                            disable_progress_bar=True)[0]

  with torch.no_grad():
      logits = torch.zeros(len(definitions), dtype=torch.double).to(DEVICE)
      for i, bert_input in list(enumerate(features)):
          logits[i] = model.ranking_linear(
              model.bert(
                  input_ids=torch.tensor(bert_input.input_ids, dtype=torch.long).unsqueeze(0).to(DEVICE),
                  attention_mask=torch.tensor(bert_input.input_mask, dtype=torch.long).unsqueeze(0).to(DEVICE),
                  token_type_ids=torch.tensor(bert_input.segment_ids, dtype=torch.long).unsqueeze(0).to(DEVICE)
              )[1]
          )
      scores = softmax(logits, dim=0)

      preds = (sorted(zip(sense_keys, definitions, scores), key=lambda x: x[-1], reverse=True))

  sense = preds[0][0]
  meaning = preds[0][1]
  return sense

In [31]:
import statistics 
from statistics import mode 
import re

def get_synsets_for_word (word):
  return set(wn.synsets(word))
  
keyword_best_sense = {}

for keyword in  keyword_sentence_mapping:
  print ("\n\n")
  print("Original word: ",keyword)
  try:
    identified_synsets=get_synsets_for_word(keyword)
  except:
    continue
  for synset in identified_synsets:
    print (synset,"   ",synset.definition())
  top_3_sentences = keyword_sentence_mapping[keyword][:3]
  best_senses=[]
  for sent in top_3_sentences:
    insensitive_keyword = re.compile(re.escape(keyword), re.IGNORECASE)
    modified_sentence = insensitive_keyword.sub(" [TGT] "+keyword+" [TGT] ", sent,count=1)
    modified_sentence = " ".join(modified_sentence.split())
    print ("modified sentence ",modified_sentence)
    best_sense = get_sense(modified_sentence)
    best_senses.append(best_sense)
  best_sense = mode(best_senses)
  print ("Best sense: ",best_sense)
  defn = best_sense.definition()
  print (defn)
  keyword_best_sense [keyword] = defn




Original word:  plate
Synset('plate.n.07')     the thin under portion of the forequarter
Synset('denture.n.01')     a dental appliance that artificially replaces missing teeth
Synset('plate.n.14')     a metal sheathing of uniform thickness (such as the shield attached to an artillery piece to protect the gunners)
Synset('plate.n.08')     a main course served on a plate
Synset('plate.n.03')     a full-page illustration (usually on slick paper)
Synset('plate.n.09')     any flat platelike body structure or part
Synset('plate.n.04')     dish on which food is served or from which food is eaten
Synset('plate.n.02')     a sheet of metal or wood or glass or plastic
Synset('plate.n.10')     the positively charged electrode in a vacuum tube
Synset('plate.n.06')     a rigid layer of the Earth's crust that is believed to drift slowly
Synset('home_plate.n.01')     (baseball) base consisting of a rubber slab where the batter stands; it must be touched by a base runner in order to score
Synset('pl

In [32]:
pprint (keyword_best_sense)

{'boundaries': 'the line or plane indicating the limit or extent of something',
 'crust': 'the outer layer of the Earth',
 'divergent': 'tending to move apart in different directions',
 'molten': 'reduced to liquid form by heating',
 'pacific': 'relating to or bordering the Pacific Ocean',
 'plate': "a rigid layer of the Earth's crust that is believed to drift slowly",
 'rock': 'material consisting of the aggregate of minerals like those making '
         "up the Earth's crust",
 'volcanoes': "a fissure in the earth's crust (or in the surface of some other "
              'planet) through which molten lava and gases erupt'}


In [33]:
import random 
from prettytable import PrettyTable
x = PrettyTable()
all_keywords= list(keyword_best_sense.keys())
all_definitions = list(keyword_best_sense.values())
random.shuffle(all_keywords)
random.shuffle(all_definitions)
print (all_keywords)
print (all_definitions)

['molten', 'rock', 'divergent', 'plate', 'pacific', 'volcanoes', 'crust', 'boundaries']
['tending to move apart in different directions', "a rigid layer of the Earth's crust that is believed to drift slowly", "material consisting of the aggregate of minerals like those making up the Earth's crust", 'relating to or bordering the Pacific Ocean', "a fissure in the earth's crust (or in the surface of some other planet) through which molten lava and gases erupt", 'the line or plane indicating the limit or extent of something', 'the outer layer of the Earth', 'reduced to liquid form by heating']


In [34]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

x.field_names=['Word', "Definition"]
for word,defn in zip(all_keywords,all_definitions):
  x.add_row([word,defn])

printmd("**Match the following words to their correct meanings.**")
# print ("\n")
print (x)

**Match the following words to their correct meanings.**

+------------+-------------------------------------------------------------------------------------------------------------------+
|    Word    |                                                     Definition                                                    |
+------------+-------------------------------------------------------------------------------------------------------------------+
|   molten   |                                   tending to move apart in different directions                                   |
|    rock    |                        a rigid layer of the Earth's crust that is believed to drift slowly                        |
| divergent  |              material consisting of the aggregate of minerals like those making up the Earth's crust              |
|   plate    |                                     relating to or bordering the Pacific Ocean                                    |
|  pacific   | a fissure in the earth's crust (or in the surface of some other plan